In [ ]:
from interaction_inference import simulation, dataset
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import tqdm
import numpy as np

# Effect of interaction & telegraph on distribution

Plot the distribution of counts as interaction / telegraph effects are applied to a birth-death / poisson model to illustrate changes. Try to understand why probabilities are so good at detecting telegraph and moments are so good at detecting interaction.

In [ ]:
interaction_values = [1, 2, 3]

# number of pairs
gene_pairs = len(interaction_values)
cells = 1000
rate = 5
gene = 1

# dataframes
params_df = pd.DataFrame(index=[f"Gene-pair-{i}" for i in range(gene_pairs)], columns=['k_on_1', 'k_on_2', 'k_off_1', 'k_off_2', 'k_tx_1', 'k_tx_2', 'k_deg_1', 'k_deg_2', 'k_reg'])
counts_df = pd.DataFrame(index=[f"Gene-pair-{i}" for i in range(gene_pairs)], columns=[f"Cell-{j}" for j in range(cells)])

# for each gene
for i in tqdm.tqdm(range(gene_pairs)):

    # Set reaction rate parameters
    k_on_1 = gene
    k_on_2 = gene
    k_off_1 = 1 - gene
    k_off_2 = 1 - gene
    k_tx_1 = rate
    k_tx_2 = rate
    k_deg_1 = 1
    k_deg_2 = 1
    k_reg = interaction_values[i]

    # store parameters
    params_df.iloc[i] = [k_on_1, k_on_2, k_off_1, k_off_2, k_tx_1, k_tx_2, k_deg_1, k_deg_2, k_reg]

    params = {
        'k_on_1': k_on_1,
        'k_on_2': k_on_2,
        'k_off_1': k_off_1,
        'k_off_2': k_off_2,
        'k_tx_1': k_tx_1,
        'k_tx_2': k_tx_2,
        'k_deg_1': k_deg_1,
        'k_deg_2': k_deg_2,
        'k_reg': k_reg
    }

    # simulate sample from model
    sample = simulation.gillespie_telegraph(params, cells)

    # store counts
    counts_df.iloc[i] = sample

# construct dataset object
data = dataset.Dataset(name)

# store information
data.count_dataset = counts_df
data.param_dataset = params_df
data.cells = cells
data.gene_pairs = len(interaction_values)
data.beta = np.array([1.0 for j in range(cells)])
